In [2]:
import pandas as pd
import numpy as np
import random

# read the MNIST data
testx = np.loadtxt("testx.csv", delimiter=',', dtype=int)
testy = np.loadtxt("testy.csv", delimiter=',', dtype=int)
trainx = np.loadtxt("trainx.csv", delimiter=',', dtype=int)
trainy = np.loadtxt("trainy.csv", delimiter=',', dtype=int)

L = 3                 # number of layers including input and output
sizes = [784, 30, 10] # number of neurons in each layer

# the sigmoid activation function
def f(z):
    return (1 / (1 + np.exp(-z)))

# sigmoid derivagtive function from sigmoid quiz
def fprime(z):
    return f(z) * (1 - f(z))

# convert a digit d to a 10-element vector
# e.g. 6 is converted to [0,0,0,0,0,0,1,0,0,0]
def digit2vector(d):
    return np.concatenate((np.repeat(0, d), [1], np.repeat(0, 9-d)))

# a feedforward function that returns the activations
# from each layer and the weighted inputs to each layer
# so that they can be used during backpropagation.
# W,b contain the weights, biases in the network.
# x is the input of a single training example (a vector of length 784).
def feedforward(W, b, x):
    a = [x, np.zeros(sizes[1]), np.zeros(sizes[2])] # x is input layer
    z = [x, np.zeros(sizes[1]), np.zeros(sizes[2])]

    z[1] = np.dot(W[0], x) + b[0] #weighted input layer 1
    a[1] = f(z[1]) #activation layer 1 using sigmoid
    z[2] = np.dot(W[1] , a[1]) + b[1] #weighted input layer 2
    a[2] = f(z[2]) #activation layer 2 using sigmoid
    return a, z 

# given an input vector, return the predicted digit
def classify(W, b, x):
    a,z = feedforward(W,b,x)
    h = a[2] # h is the column of values mapped from sigmoid function
    return np.argmax(h) # the index of the largest value is the number classification


# helper function for backprop().
# this function computes the error for a single training example.
# W contains the weights in the network.
# a contains the activations.
# z contains the weighted inputs.
# y is the correct digit.
# returns δ = the error. the size of δ is [ 784, 30, 10 ]
def compute_error(W, a, z, y):
    δ = [np.zeros(sizes[0]), np.zeros(sizes[1]), np.zeros(sizes[2])]
    # note that δ[1] is junk. we put it there so that the indices make sense.

    # at the output layer L
    δ[2] = -(digit2vector(y) - a[2]) * fprime(z[2])

    # for each earlier layer L-1,L-2,..,2 (for the HW, this means only layer 2)
    δ[1] = np.dot(W[1].T, δ[2]) * fprime(z[1])

    return δ


# helper function for backprop(). given the errors δ and the
# activations a for a single training example, this function returns
# the gradient components ∇W and ∇b.
# this function implements the equations BP3 and BP4.
def compute_gradients(δ, a):
    grad_W = [np.zeros((sizes[1], sizes[0])), np.zeros((sizes[2], sizes[1]))]
    grad_b = [np.zeros(sizes[1]), np.zeros(sizes[2])]
    #grad_b is δ[1],δ[2] from derivative proof
    grad_b[0] = δ[1]
    grad_b[1] = δ[2]
    #grad_W is δ[1]*a[0],δ[2]*a[1] from question 2 of homework proof
    grad_W[0] = np.outer(δ[1] , a[0]) #np.outer ensure that this results in a 30x784 matrix
    grad_W[1] = np.outer(δ[2] , a[1]) #results in a 10x30 matrix
    return grad_W, grad_b

# backpropagation. returns ∇W and ∇b for a single training example.
def backprop(W, b, x, y):
    (a, z) = feedforward(W, b, x)
    δ = compute_error(W, a, z, y)
    (grad_W, grad_b) = compute_gradients(δ, a)
    return grad_W, grad_b


# gradient descent algorithm.
# W = weights in the network
# b = biases in the network
# batch = the indices of the observations in the batch, i.e. the rows of trainx
# α = step size
# λ = regularization parameter
def GD(W, b, batch, α=0.01, λ=0.01):
    m = len(batch)    # batch size

    # data structure to accumulate the sum over the batch.
    # in the lecture notes as well as Ng's article, sumW is ΔW and sumb is Δb.
    sumW = [ np.zeros((sizes[1], sizes[0])),
             np.zeros((sizes[2], sizes[1])) ]
    sumb = [ np.zeros(sizes[1]), np.zeros(sizes[2])]

    # for each training example in the batch, use backprop
    # to compute the gradients and add them to the sum
    for i in range(m):
        grad_W, grad_b = backprop(W,b,trainx[batch[i],:], trainy[batch[i]]) #index of training data x and y is from batch indicies, first index from batch used to select the row from trainx
        sumW[0] += grad_W[0] #each index of gradient is added to the sum
        sumW[1] += grad_W[1]
        sumb[0] += grad_b[0]
        sumb[1] += grad_b[1]
    
    # make the update to the weights and biases and take a step
    # of gradient descent. note that we use the average gradient.

    #each index of W and b is updated with the step and average gradient (and regularization term for weights)
    W[0] = W[0] - α * (((1/m)*sumW[0]) + (λ * W[0]))
    W[1] = W[1] - α * (((1/m)*sumW[1]) + (λ * W[1]))
    b[0] = b[0] - α * ((1/m) * sumb[0])
    b[1] = b[1] - α * ((1/m) * sumb[1])

    # return the updated weights and biases. we also return the gradients

    return W, b, grad_W, grad_b


# classify the test data and compute the classification accuracy
def accuracy(W, b): 
    ntest = len(testy)
    yhat = np.zeros(ntest)
    for i in range(ntest):
        yhat[i] = classify(W, b, testx[i,:])
    hit_rate =  np.sum(testy == yhat) / ntest # hit rate
    return hit_rate


# train the neural network using batch gradient descent.
# this is a driver function to repeatedly call GD().
# N = number of observations in the training data.
# m = batch size
# α = learning rate (i.e. step size)
# λ = regularization parameter
def BGD(N, m, epochs, α=0.01, λ=0.01):
    sizes = [784, 30, 10]
    #random initialization of the weights and biases
    #python uses np.random.normal to directly generate random numbers from a normal distribution
    W = [ np.random.normal(0,1, (sizes[1], sizes[0])),  # layer 1 to 2
          np.random.normal(0,1, (sizes[2], sizes[1])) ] # layer 2 to 3
    b = [ np.random.normal(0,1, sizes[1]),   # layer 2
          np.random.normal(0,1, sizes[2]) ]  # layer 3
    
    grad_W = [ np.zeros((sizes[1], sizes[0])),  # layer 1 to 2
          np.zeros((sizes[2], sizes[1])) ] # layer 2 to 3
    grad_b = [ np.zeros(sizes[1]),   # layer 2
          np.zeros(sizes[2]) ]   # layer 3

    
    for i in range(epochs): 
        remaining = list(range(N)) #convert to list of 0:N-1 to track indicies
        while len(remaining) > 0: # while there are still observations to create batches from 
            batch = random.sample(remaining, m) #samples batch of m from the remaining indicies
            for index in batch:
                remaining.remove(index) #removes the batches indicies from remaining
            W,b,grad_W,grad_b = GD(W,b,batch)
        print(f"epoch: {i + 1} is done. Accuracy = {accuracy(W,b)}")

    return W, b, grad_W, grad_b


# some tuning parameters
N = len(trainy)
m = 25       # batch size
epochs = 30  # number of complete passes through the training data
α = 0.01     # learning rate / step size
λ = 0.01     # regularization parameter
W, b, grad_W, grad_b = BGD(N, m, epochs, α=α, λ=λ)
print()
print("Classification accuracy: ")
print(accuracy(W,b))



C:\Users\angie\AppData\Local\Temp\ipykernel_20976\1505340667.py:17: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-z)))


epoch: 1 is done. Accuracy = 0.1203
epoch: 2 is done. Accuracy = 0.1555
epoch: 3 is done. Accuracy = 0.2123
epoch: 4 is done. Accuracy = 0.3025
epoch: 5 is done. Accuracy = 0.3687
epoch: 6 is done. Accuracy = 0.4188
epoch: 7 is done. Accuracy = 0.4918
epoch: 8 is done. Accuracy = 0.5806
epoch: 9 is done. Accuracy = 0.6498
epoch: 10 is done. Accuracy = 0.7297
epoch: 11 is done. Accuracy = 0.7998
epoch: 12 is done. Accuracy = 0.8395
epoch: 13 is done. Accuracy = 0.8646
epoch: 14 is done. Accuracy = 0.8754
epoch: 15 is done. Accuracy = 0.8929
epoch: 16 is done. Accuracy = 0.8998
epoch: 17 is done. Accuracy = 0.9026
epoch: 18 is done. Accuracy = 0.905
epoch: 19 is done. Accuracy = 0.9061
epoch: 20 is done. Accuracy = 0.9056
epoch: 21 is done. Accuracy = 0.9055
epoch: 22 is done. Accuracy = 0.9051
epoch: 23 is done. Accuracy = 0.9072
epoch: 24 is done. Accuracy = 0.9087
epoch: 25 is done. Accuracy = 0.9049
epoch: 26 is done. Accuracy = 0.908
epoch: 27 is done. Accuracy = 0.9086
epoch: 28 is